In [3]:
import os
import pymssql

import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

In [6]:
DATABASE_URL_PROCESSO = f"mssql+pymssql://{os.getenv('SQL_SERVER_USER')}:{os.getenv('SQL_SERVER_PASS')}@{os.getenv('SQL_SERVER_HOST')}/processo"

def get_connection():
    """Establish a connection to the SQL Server database.
    Returns:
        pymssql.Connection: A connection object to the SQL Server database.
    """
    return pymssql.connect(
        server=os.getenv('SQL_SERVER_HOST'),
        user=os.getenv('SQL_SERVER_USER'),
        password=os.getenv('SQL_SERVER_PASS'),
        database='processo'
    )

In [ ]:
numero_processo = '008348'
ano_processo = 2016

query = f"""
SELECT p.idprocesso as id_processo, NumeroProcesso as numero_processo, AnoProcesso as ano_processo, 
 IdComposicaoPauta as id_composicao_pauta, idVotoPauta as id_voto_pauta, 
numero_sessao, ano_sessao, DataSessao as data_sessao,  
Relatorio as relatorio, FundamentacaoVoto as fundamentacao_voto, 
Conclusao as conclusao, texto_acordao, o.nome as orgao_responsavel, o.IdOrgao as id_orgao_responsavel,
gp.Nome as nome_responsavel, gp.Documento as documento_responsavel, gp.TipoPessoa as tipo_responsavel, gp.idpessoa as id_pessoa
FROM processo.dbo.vw_ia_votos_acordaos_decisoes ia 
LEFT JOIN processo.dbo.processos p ON ia.NumeroProcesso = p.numero_processo  AND ia.AnoProcesso = p.ano_processo 
LEFT JOIN processo.dbo.Orgaos o ON o.IdOrgao = p.IdOrgaoEnvolvido
LEFT JOIN processo.dbo.Pro_ProcessosResponsavelDespesa pprd ON p.IdProcesso = pprd.IdProcesso 
LEFT JOIN processo.dbo.GenPessoa gp ON pprd.IdPessoa = gp.IdPessoa 
WHERE p.numero_processo = {numero_processo}
AND p.ano_processo = {ano_processo}
"""

In [24]:
def get_df_decisao(numero_processo, ano_processo):
    query = f"""
    SELECT p.idprocesso as id_processo, NumeroProcesso as numero_processo, AnoProcesso as ano_processo, 
    IdComposicaoPauta as id_composicao_pauta, idVotoPauta as id_voto_pauta, 
    numero_sessao, ano_sessao, DataSessao as data_sessao,  
    Relatorio as relatorio, FundamentacaoVoto as fundamentacao_voto, 
    Conclusao as conclusao, texto_acordao, o.nome as orgao_responsavel, o.IdOrgao as id_orgao_responsavel,
    gp.Nome as nome_responsavel, gp.Documento as documento_responsavel, gp.TipoPessoa as tipo_responsavel, gp.idpessoa as id_pessoa
    FROM processo.dbo.vw_ia_votos_acordaos_decisoes ia 
    LEFT JOIN processo.dbo.processos p ON ia.NumeroProcesso = p.numero_processo  AND ia.AnoProcesso = p.ano_processo 
    LEFT JOIN processo.dbo.Orgaos o ON o.IdOrgao = p.IdOrgaoEnvolvido
    LEFT JOIN processo.dbo.Pro_ProcessosResponsavelDespesa pprd ON p.IdProcesso = pprd.IdProcesso 
    LEFT JOIN processo.dbo.GenPessoa gp ON pprd.IdPessoa = gp.IdPessoa 
    WHERE p.numero_processo = {numero_processo}
    AND p.ano_processo = {ano_processo}
    """

    df = pd.read_sql(query, get_connection())

    group_cols = [
            'id_processo', 'numero_processo', 'ano_processo', 'id_composicao_pauta', 
            'id_voto_pauta', 'numero_sessao', 'ano_sessao', 'data_sessao', 'relatorio',
            'fundamentacao_voto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel',
    ]

    # Define as colunas para criar o dicionário de pessoas
    person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

    # Agrupa o DataFrame e aplica uma função lambda para criar a lista de dicionários
    df = df.groupby(group_cols, dropna=False).apply(
        lambda x: pd.Series({'responsaveis': x[person_cols].apply(
            lambda y: y.dropna().to_dict(), axis=1
        ).tolist()})
    ).reset_index()


    df= df[['id_composicao_pauta', 'id_voto_pauta']].merge(df, on=['id_composicao_pauta', 'id_voto_pauta'], how='left')
    df['data_sessao'] = pd.to_datetime(df['data_sessao'], errors='coerce')

    return df

In [25]:
numero_processo = '008348'
ano_processo = 2016

df = get_df_decisao(numero_processo, ano_processo)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_7188\1212765339.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())
C:\Users\05911205424\AppData\Local\Temp\ipykernel_7188\1212765339.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(group_cols, dropna=False).apply(


In [26]:
df

,id_composicao_pauta,id_voto_pauta,id_processo,numero_processo,ano_processo,numero_sessao,ano_sessao,data_sessao,relatorio,fundamentacao_voto,conclusao,texto_acordao,orgao_responsavel,id_orgao_responsavel,responsaveis
0,118008,42430,412889,008348,2016,0017V,2024,2024-11-25 07:00:00,Conforme documentado pela DIRETORIA DE ATOS E ...,Exaro juízo positivo para presidir a relatoria...,Com fundamento no parecer ministerial apensado...,"Vistos, relatados e discutidos estes autos, co...",TRIBUNAL DE CONTAS DO ESTADO,532,"[{'nome_responsavel': 'MANOEL LINO BEZERRA', '..."


In [21]:
df = pd.read_sql(query, get_connection())

group_cols = [
        'id_processo', 'numero_processo', 'ano_processo', 'id_composicao_pauta', 
        'id_voto_pauta', 'numero_sessao', 'ano_sessao', 'data_sessao', 'relatorio',
        'fundamentacao_voto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel',
]

# Define as colunas para criar o dicionário de pessoas
person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

# Agrupa o DataFrame e aplica uma função lambda para criar a lista de dicionários
df = df.groupby(group_cols, dropna=False).apply(
    lambda x: pd.Series({'responsaveis': x[person_cols].apply(
        lambda y: y.dropna().to_dict(), axis=1
    ).tolist()})
).reset_index()


df_ob = df[['id_composicao_pauta', 'id_voto_pauta']].merge(df, on=['id_composicao_pauta', 'id_voto_pauta'], how='left')
df_ob['data_sessao'] = pd.to_datetime(df_ob['data_sessao'], errors='coerce')

C:\Users\05911205424\AppData\Local\Temp\ipykernel_7188\1623314607.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())
C:\Users\05911205424\AppData\Local\Temp\ipykernel_7188\1623314607.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(group_cols, dropna=False).apply(


In [23]:
df_ob

,id_composicao_pauta,id_voto_pauta,id_processo,numero_processo,ano_processo,numero_sessao,ano_sessao,data_sessao,relatorio,fundamentacao_voto,conclusao,texto_acordao,orgao_responsavel,id_orgao_responsavel,responsaveis
0,118008,42430,412889,008348,2016,0017V,2024,2024-11-25 07:00:00,Conforme documentado pela DIRETORIA DE ATOS E ...,Exaro juízo positivo para presidir a relatoria...,Com fundamento no parecer ministerial apensado...,"Vistos, relatados e discutidos estes autos, co...",TRIBUNAL DE CONTAS DO ESTADO,532,"[{'nome_responsavel': 'MANOEL LINO BEZERRA', '..."
